# Tarea 2: implementar un modelo de inferencia en tiempo real

## Tarea 2.1: configuración del entorno

Instale los paquetes y las dependencias.

In [ ]:
#install-dependencies
import boto3
import pandas as pd
import sagemaker
import sagemaker_datawrangler
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
prefix = 'sagemaker/mlasms'
bucket = sagemaker.Session().default_bucket()
s3_client = boto3.client("s3")

Revise el conjunto de datos procesados del cliente.

In [ ]:
#explore-dataset
column_list = ['income','age','workclass','education','education_num','marital_status','occupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week']
lab_test_data = pd.read_csv('adult_data_processed.csv', names=(column_list), header=1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.dtypes
lab_test_data.head()

Guarde el modelo del laboratorio de entrenamiento y ajuste en el bucket predeterminado de Amazon Simple Storage Service (Amazon S3). Configure un modelo usando **create_model** y configure **ModelDataUrl** para referenciar al modelo entrenado.

In [ ]:
#set-up-model
# Upload the model to your Amazon S3 bucket
s3_client.upload_file(
    Filename="model.tar.gz", Bucket=bucket, Key=f"{prefix}/models/model.tar.gz"
)

# Set a date to use in the model name
create_date = time.strftime("%Y-%m-%d-%H-%M-%S")
model_name = 'income-model-{}'.format(create_date)

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)

# Set up the model
income_model = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': f's3://{bucket}/{prefix}/models/model.tar.gz',
    }
)

## Tarea 2.2: crear un punto de enlace a partir del modelo reentrenado y sintetizado que se proporciona

Puede crear un punto de enlace con el SDK para Python de Amazon SageMaker en tres pasos:
1. Cree un modelo de SageMaker.
2. Cree una configuración para un punto de enlace HTTPS.
3. Cree un punto de enlace HTTPS.

Consulte [Cree su punto de enlace e implemente su modelo](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html) para obtener más información sobre cómo crear un punto de enlace.

Ya creó un modelo. Ahora está listo para crear un punto de enlace y su configuración. 

Primero, establezca la configuración del punto de enlace y el tipo de instancia que desea usar. Luego, llame a la API CreateEndpointConfig.

Para crear una configuración del punto de enlace, debe establecer las siguientes opciones:
- **VariantName**: el nombre de la variante de producción (uno o más modelos en producción)
- **ModelName**: el nombre del modelo que desea alojar. Este es el nombre que especificó cuando creó el modelo.
**InstanceType**: es el tipo de instancia de cómputo.
- **InitialInstanceCount**: la cantidad de instancias que se deben iniciar en un primer momento.

Para registrar las entradas en su punto de enlace y las salidas de la inferencia de los puntos de enlace en tiempo real de SageMaker en Amazon S3, puede habilitar una función llamada Captura de datos. La Captura de datos se usa habitualmente para registrar información que se puede utilizar para el entrenamiento, la depuración y la supervisión. Cuando explore su punto de enlace en Amazon SageMaker Studio, se mostrará información más detallada sobre el punto de enlace si está habilitada la Captura de datos. La configuración de la Captura de datos se presenta más adelante en este laboratorio para que aprenda a habilitarla.

Consulte [Captura de datos](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-data-capture.html) para obtener más información sobre cómo agregar la Captura de datos.

In [ ]:
#create-endpoint-configuration 
# Create an endpoint config name. Here you create one based on the date  
# so it you can search endpoints based on creation time.
endpoint_config_name = 'income-model-real-time-endpoint-{}'.format(create_date)                              
instance_type = 'ml.m5.xlarge'   
initial_sampling_percentage = 25 # Choose a value between 0 and 100
capture_modes = [ "Input",  "Output" ] # Specify input, output, or both

endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ],
    DataCaptureConfig= {
        'EnableCapture': True, # Whether data should be captured or not.
        'InitialSamplingPercentage' : initial_sampling_percentage,
        'DestinationS3Uri': f's3://{bucket}/data-capture',
        'CaptureOptions': [{"CaptureMode" : capture_mode} for capture_mode in capture_modes]
    }
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

A continuación, cree un punto de enlace. Cuando crea un punto de enlace en tiempo real, SageMaker inicia las instancias de cómputo de machine learning (ML) e implementa uno o más modelos según lo especificado en la configuración. En este laboratorio, solo implementa un modelo para inferencias. En SageMaker, puede crear un punto de enlace de varios modelos. Consulte [Invocar un punto de enlace de varios modelos](https://docs.aws.amazon.com/sagemaker/latest/dg/invoke-multi-model-endpoint.html) para obtener más información sobre puntos de enlace de varios modelos.

Cuando el punto de enlace esté en funcionamiento, la función auxiliar imprimirá el Amazon Resource Name (ARN) del punto de enlace. La creación del punto de enlace demorará entre 3 y 7 minutos.

In [ ]:
#create-endpoint
# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = '{}-name'.format(endpoint_config_name)

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name
) 

def wait_for_endpoint_creation_complete(endpoint):
    """Helper function to wait for the completion of creating an endpoint"""
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    status = response.get("EndpointStatus")
    while status == "Creating":
        print("Waiting for Endpoint Creation")
        time.sleep(15)
        response = sm.describe_endpoint(EndpointName=endpoint_name)
        status = response.get("EndpointStatus")

    if status != "InService":
        print(f"Failed to create endpoint, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create endpoint {create_endpoint_response['EndpointArn']}, status: {status}, reason: {failureReason}"
        )
    print(f"Endpoint {create_endpoint_response['EndpointArn']} successfully created.")

wait_for_endpoint_creation_complete(endpoint=create_endpoint_response)


En SageMaker Studio, puede revisar los detalles del punto de enlace debajo de la pestaña **Endpoints** (Puntos de enlace).

El próximo paso abre una nueva pestaña en SageMaker Studio. Para seguir esas instrucciones, utilice las siguientes opciones:
- **Opción 1**: ver las pestañas una al lado de la otra. Para crear una vista de pantalla dividida de la ventana principal de SageMaker Studio, arrastre la pestaña **real_time_inference.ipynb** hacia el lado o seleccione (con el botón derecho del mouse) la pestaña **real_time_inference.ipynb** y elija **New View for Notebook** (Nueva vista para el cuaderno). Ahora puede tener las instrucciones visibles mientras explora el punto de enlace.
- **Opción 2**: alternar entre las pestañas de SageMaker Studio para seguir estas instrucciones. Cuando termine de explorar el punto de enlace, seleccione la pestaña **real_time_inference.ipynb** para volver al cuaderno.

1. Seleccione el ícono de **SageMaker Home** (Inicio de SageMaker).
2. Seleccione **Deployments** (Implementaciones).
3. Seleccione **Endpoints** (Puntos de enlace).

Verá la pestaña **Endpoints** (Puntos de enlace) en SageMaker Studio.

4. Seleccione el punto de enlace que tenga **income-model-real-time-** en la columna **Name** (Nombre).

Si el punto de enlace no aparece, seleccione el ícono de actualización hasta que aparezca en la lista.

Verá la pestaña **ENDPOINT DETAILS** (DETALLES DEL PUNTO DE ENLACE) en SageMaker Studio.

5. Vaya a la pestaña **AWS settings** (Configuración de AWS).

Si abrió el punto de enlace antes de que terminara de crearse, seleccione el ícono de actualización hasta que el valor de **Endpoint status** (Estado del punto de enlace) cambie de *Creating* (Creando) a *InService* (En funcionamiento).

En **Endpoint type** (Tipo de punto de enlace), verá **Real-time** (En tiempo real). Las secciones **Data capture settings** (Configuración de la captura de datos) y **Endpoint configuration settings** (Ajustes de la configuración de punto de enlace) muestran las configuraciones que seleccionó antes en el cuaderno.

## Tarea 2.3: invocar un punto de enlace para una inferencia en tiempo real con registros de clientes en tiempo real

Después de que implemente su modelo con los servicios de alojamiento de SageMaker, puede enviarle datos de prueba al modelo en ese punto de enlace para probarlo.

Tiene varios registros de clientes que sabe que perciben un ingreso mayor que o igual a 50 000 USD (un valor de **ingresos** de **1**) y varios que perciben un ingreso inferior a ese monto (un valor de **ingresos** de **0**). Invoque el punto de enlace con estos registros y vea las puntuaciones que se muestran.

Para ver las predicciones en tiempo real del punto de enlace, lea el texto del cuerpo que se muestra en la respuesta, que contiene una lista de puntuaciones de las predicciones. La puntuación para cada registro varía de **0** a **1**. Los números cercanos a **1** indican que es más probable que esos clientes tengan un ingreso mayor que o igual a 50 000 USD. Por ejemplo, es más probable que un cliente con una puntuación de predicción de **0,42** tenga un ingreso mayor que o igual a 50 000 USD que un cliente cuya puntuación de predicción es de **0,14**.

In [ ]:
#invoke-endpoint-real-time-records
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('56,3,6,6,0,3,1,0,0,1,0,13\n' +
                '29,2,2,2,0,1,0,0,0,0,0,70\n' +
                '79,0,1,1,0,3,5,0,0,0,0,20\n', 'utf-8')
)

print(response)

print('\nTesting with records that have an income value of 1:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

response = sagemaker_runtime.invoke_endpoint(
    ContentType='text/csv',
    EndpointName=endpoint_name, 
    Body=bytes('19,0,1,1,1,3,2,0,0,0,0,32\n' +
                '31,0,1,1,2,1,2,1,1,0,0,40\n' +
                '23,0,1,1,1,0,1,0,0,0,0,40\n', 'utf-8')
)

print('\nTesting with records that have an income value of 0:')
print('The returned scores are: {}'.format(response['Body'].read().decode('utf-8')))

## Tarea 2.4: eliminar el punto de enlace

La eliminación de un punto de enlace se puede lograr en tres pasos. Primero, elimine el punto de enlace. Luego, elimine la configuración del punto de enlace. Por último, si ya no necesita el modelo que implementó, elimínelo.

In [ ]:
#delete-resources
# Delete endpoint
sm.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
                   
# Delete model
sm.delete_model(ModelName=model_name)

### Conclusión

¡Felicitaciones! Usó SageMaker para crear correctamente un punto de enlace en tiempo real, con el SDK para Python de SageMaker, y para invocarlo.

La próxima tarea del laboratorio se centra en la implementación de un modelo de inferencias usando una inferencia sin servidor.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Tarea 3: implementar un modelo de inferencia sin servidor**.